In [7]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# TODO get interactive plotting working in jupyterlab
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sympy as S 

from interval import interval, inf, imath

from safe_train import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
projection_training_loop()

2023-06-20 12:58:23.970305: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-06-20 12:58:23.981827: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.



Start of epoch 0
[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-179.58066]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-12.600288], dtype=float32)>]
[<tf.Variable 'dense_7/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.44550288]], dtype=float32)>, <tf.Variable 'dense_7/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]
safe region test passed, interval was interval([11.010058015584946, 21.920116037130356])

Start of epoch 1
[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[50.959656]], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-6.4002867], dtype=float32)>]
[<tf.Variable 'dense_7/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[0.5455029]], dtype=float32)>, <tf.Variable 'dense_7/bias:0' shape=(1,) dtype=float32, numpy=array([0.09999999], dtype=float32)>]
safe region test passed, interval was interval([11.986208885908127, 23.778968781232834])

Start of epoch 2
[<tf.Tensor: shape=(1, 1), dty

In [ ]:
projection_training_loop()

In [68]:
# output_interval = interval[9.711857616901398, 19.23788982629776]
weight = 0.4763016
bias = 0.18582541
input_interval = interval[20, 40]
input_interval * weight + bias

# goal interval: [10, 30]
goal_interval = interval[10, 30]

In [69]:
# output_interval = interval[9.711857616901398, 19.23788982629776]
# from wolfram alpha!
new_weight = 0.490664
new_bias = 0.186733
input_interval * new_weight + new_bias

interval([10.000013, 19.813292999999998])

In [46]:
theta = np.array([weight, bias])
theta

array([0.4763016 , 0.18582541])

**insight**: `input_interval` COULD be anything, from the penultimate layer output - for single-node, it's the actual input

In [13]:
w, b = S.symbols("w b")

In [23]:
input_interval[0].inf

20.0

In [24]:
input_interval[0].sup

40.0

In [25]:
# symbolic interval

In [28]:
lower = input_interval[0].inf * w + b
lower

b + 20.0*w

In [30]:
upper = input_interval[0].sup * w + b
upper

b + 40.0*w

In [50]:
-input_interval[0].inf

-20

In [ ]:
shift_lower = np.array([0, goal_interval[0].inf])
direction_lower = np.array([1, -input_interval[0].inf])
project_lower = (direction_lower @ (theta - shift_lower)) / (direction_lower @ direction_lower) * direction_lower
param_lower = project_lower + shift_lower

shift_upper = np.array([0, goal_interval[0].sup])
direction_upper = np.array([1, -input_interval[0].sup])
project_upper = (direction_upper @ (theta - shift_upper)) / (direction_upper @ direction_upper) * direction_upper
param_upper = project_upper + shift_upper

best_param = min([param_upper, param_lower], key = lambda param: np.linalg.norm(theta - param))

In [56]:
shift_lower = np.array([0, goal_interval[0].inf])
shift_lower

array([ 0., 10.])

In [57]:
direction_lower = np.array([1, -input_interval[0].inf])
direction_lower

array([  1., -20.])

In [73]:
project_lower = (direction_lower @ (theta - shift_lower)) / (direction_lower @ direction_lower) * direction_lower
project_lower

array([ 0.4906728 , -9.81345603])

In [75]:
param_lower = project_lower + shift_lower
param_lower

array([0.4906728 , 0.18654397])

In [62]:
shift_upper = np.array([0, goal_interval[0].sup])
shift_upper

array([ 0., 30.])

In [63]:
direction_upper = np.array([1, -input_interval[0].sup])
direction_upper

array([  1., -40.])

In [71]:
project_upper = (direction_upper @ (theta - shift_upper)) / (direction_upper @ direction_upper) * direction_upper
project_upper

array([  0.74518631, -29.80745247])

In [76]:
param_upper = project_upper + shift_upper
param_upper

array([0.74518631, 0.19254753])

In [77]:
np.linalg.norm(theta - param_upper)

0.2689687251525259

In [78]:
np.linalg.norm(theta - param_lower)

0.014389154284640958

In [91]:
min([param_upper, param_lower], key = lambda param: np.linalg.norm(theta - param))

array([0.4906728 , 0.18654397])

In [80]:
w, b = param_lower
w, b

(0.49067280149625936, 0.18654397007481194)

In [81]:
input_interval * w + b

interval([10.0, 19.813456029925188])